In [1]:
my_packages <- c("tidyverse", "lme4", "nlme", "readxl", "lmerTest", "dplyr")

In [2]:
install.packages("lme4")

also installing the dependency ‘nloptr’


Warning message in install.packages("lme4"):
“installation of package ‘nloptr’ had non-zero exit status”
Warning message in install.packages("lme4"):
“installation of package ‘lme4’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [3]:
lapply(my_packages, require, character.only = TRUE)

Loading required package: tidyverse

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: lme4

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lme4’”
Loading required package: nlme


Attaching package: ‘nlme’


The following object is masked from ‘package:dplyr’:

    collapse


Loading required package: readxl

Loading required package: lmerTest

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lmerTest’”


[[1]]
[1] TRUE

[[2]]
[1] FALSE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] FALSE

[[6]]
[1] TRUE

In [ ]:
# This is the start of cleaning and tidying process

In [ ]:
#List excel sheets
combined_data <- excel_sheets("2_People_untidy_MD.xlsx") %>% map_df(~read_xlsx("2_People_untidy_MD.xlsx",.))
head(combined_data)

In [ ]:
#Replace missing values with 0
combined_data[is.na(combined_data)] <- 0

In [ ]:
# Divide People_total / msq to get total ppl_msq
combined_data$ppl_msq <- (combined_data$People_total / combined_data$msq)

In [ ]:
#preview
head(combined_data)
tail(combined_data)

In [ ]:
#all BP3 and OC concentration ([UVF]) data from individual CSV files; str preview showing NA values
conc_muku <- data.frame(read.csv("conc_muku.csv"))
conc_ole <- data.frame(read.csv("conc_ole.csv"))
conc_hoku <- data.frame(read.csv("conc_hoku.csv"))
str(conc_muku)
str(conc_ole)
str(conc_hoku)

In [ ]:
#combine all [UVF] data to one data frame
conc <- rbind(conc_muku, conc_ole, conc_hoku)

In [ ]:
#replace NA values with 0; str preview
conc[is.na(conc)] <- 0
str(conc)

In [ ]:
#combine [UVF] and pple from combined_data; head preview
conc_pple <- cbind(conc,combined_data)
head(conc_pple)

In [ ]:
#remove duplicate columns, reorganize columns; head preview
conc_pple_2 <- conc_pple[,-5]
conc_pple_3 <- conc_pple_2[,-8]
conc_pple_4 <- conc_pple_3[,-6]
conc_pple_fin <- conc_pple_4 %>% relocate(Date, .after = Moon)
head(conc_pple_fin)

In [ ]:
### Kaimana_tide dataset
Diamond_data <- read.csv("Diamond_tide.csv")
str(Diamond_data)

In [ ]:
### Combining PreTide and PostTide columns into one
Diamond_data$PreTide_PostTide <- paste(Diamond_data$PreTide, Diamond_data$PostTide, sep="_")
str(Diamond_data)

In [ ]:
### Removing old columns
##download this one as csv
tide <- Diamond_data[c("Date", "Moon", "Time", "Tide", "PreTide_PostTide")]
str(tide)

In [ ]:
### Tidying up the data again (Cleaning)
tide[c('PreTide', 'PostTide')] <- str_split_fixed(tide$PreTide_PostTide, '_', 2)
str(tide)

In [ ]:
### Removing extra joint column (Cleaning)
tide_2 <- tide[c("Date", "Moon", "Time", "Tide", "PreTide", "PostTide")]
str(tide_2)

In [ ]:
### Convert Tide column from inches to meters
tide_2$Tide <- tide_2$Tide * 0.3048
str(tide_2)

In [ ]:
### Changing PreTide and PostTide columns from character to double strings
tide_2$PreTide = as.double(tide_2$PreTide)
tide_2$PostTide = as.double(tide_2$PostTide)
str(tide_2)

In [ ]:
### Convert PreTide and PostTide columns from inches to meters
tide_2$PreTide <- tide_2$PreTide * 0.3048
tide_2$PostTide <- tide_2$PostTide * 0.3048
str(tide_2)

In [ ]:
#combine [UVF] and pple data with tidal data from tide_2
Diamond_all <- cbind(conc_pple_fin,tide_2)
head(Diamond_all)

In [ ]:
#remove duplicate columns
Diamond_all_1 <- Diamond_all[,-10]
Diamond_all_2 <- Diamond_all_1[,-9]
Diamond_all_fin <- Diamond_all_2[,-8]

In [ ]:
head(Diamond_all_fin)
str(Diamond_all_fin)

In [ ]:
#Export tidy csv file
write.csv(Diamond_all_fin, "DiamondBeach_all.csv")

In [ ]:
#This is the start of mixed model creation

In [ ]:
Diamond_all_fin$ppl_msq_s <- scale(Diamond_all_fin$ppl_msq) 

In [ ]:
Diamond_all_fin$Time_s <- scale(Diamond_all_fin$Time) 

In [ ]:
Diamond_all_fin$Tide_s <- scale(Diamond_all_fin$Tide) 

In [ ]:
Diamond_all_fin$Time <- as.numeric(Diamond_all_fin$Time)

In [ ]:
Diamond_all_fin = Diamond_all_fin %>%
mutate(Moon_s = recode(Diamond_all_fin$Moon, 
                      "Muku" = 1, 
                      "Hoku" = 2, 
                      "ʻOlekūkolu" = 3))

In [ ]:
#Time of day effect on people m^2 
#P value shows effect on ppl_msq from time is significant
T_o_D_ppl_msq <- lm(ppl_msq_s ~ Time_s, data = Diamond_all_fin)
summary(T_o_D_ppl_msq)

In [ ]:
# Concentration of BP3 by time of day and ppl_msq
Time.vs.BP3 <- lmer(BP3 ~ Time_s + ppl_msq_s + (1|Moon_s), data = Diamond_all_fin)
summary(Time.vs.BP3)

In [ ]:
# Concentration of OC by time of day and ppl_msq
Time.vs.OC <- lmer(OC ~ Time_s + ppl_msq_s + (1|Moon_s), data = Diamond_all_fin)
summary(Time.vs.OC)

In [ ]:
# Concentration of BP3 off tide, time, and ppl_msq
Tide.vs.BP3 <- lmer(BP3 ~ Tide_s + Time_s + ppl_msq_s + (1|Moon_s), data = Diamond_all_fin)
summary(Tide.vs.BP3)

In [ ]:
# Concentration of OC off tide, time, and ppl_msq
Tide.vs.OC <- lmer(OC ~ Tide_s + Time_s + ppl_msq_s + (1|Moon_s), data = Diamond_all_fin)
summary(Tide.vs.OC)

In [ ]:
# Concentration of OC versus ppl_msq and time
PPL.vs.OC <- lmer(OC ~ ppl_msq + Time_s + (1|Moon_s), data = Diamond_all_fin)
summary(PPL.vs.OC)

In [ ]:
# Concentration of BP3 versus ppl_msq and time
PPL.vs.BP3 <- lmer(BP3 ~ ppl_msq + Time_s + (1|Moon_s), data = Diamond_all_fin)
summary(PPL.vs.BP3)

In [ ]:
##BP3: effect of pretide(w/ time of day interaction) 
##and posttide (with tide of day interaction) on [BP3].
head(Diamond_all_fin)

In [ ]:
BP3_pre_post <- lmer(BP3 ~ PreTide*Time_s + PostTide*Time_s + (1|Moon_s), data = Diamond_all_fin)
summary(BP3_pre_post)

In [ ]:
OC_pre_post <- lmer(OC ~ PreTide*Time_s + PostTide*Time_s + (1|Moon_s), data = Diamond_all_fin)
summary(OC_pre_post)